![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [18]:
import pandas as pd
import time

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = "./Dados/UCI HAR Dataset/features.txt"
filename_labels = "./Dados/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "./Dados/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "./Dados/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "./Dados/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "./Dados/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "./Dados/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "./Dados/UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features['nome_var'].tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features['nome_var'].tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [19]:
%%time

# Criando o modelo de Árvore de Decisão
tree_model = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)

# Treinando o modelo com os dados de treinamento
start_time = time.time()
tree_model.fit(X_train, y_train['cod_label'])
training_time = time.time() - start_time

# Avaliando a acurácia nos dados de treinamento
train_accuracy = tree_model.score(X_train, y_train['cod_label'])

# Avaliando a acurácia nos dados de teste
test_accuracy = tree_model.score(X_test, y_test['cod_label'])

print("Acurácia nos dados de treinamento:", round(train_accuracy, 2))
print("Acurácia nos dados de teste:", round(test_accuracy, 2))
print("Tempo de treinamento:", round(training_time, 2), "segundos")
print("-" * 60)

Acurácia nos dados de treinamento: 0.98
Acurácia nos dados de teste: 0.88
Tempo de treinamento: 9.01 segundos
------------------------------------------------------------
CPU times: total: 6.02 s
Wall time: 9.05 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [20]:
%%time

# Realizando a Análise de Componentes Principais (PCA) com uma componente
n_components = 1
pca = PCA(n_components=n_components)

# Ajustando e transformando os dados de treinamento e teste com PCA
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Criando o modelo de Árvore de Decisão com a única componente PCA
tree_model_pca = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)

# Treinando o modelo com os dados de treinamento transformados
start_time = time.time()
tree_model_pca.fit(X_train_pca, y_train['cod_label'])
training_time = time.time() - start_time

# Avaliando a acurácia nos dados de treinamento transformados
train_accuracy_pca = tree_model_pca.score(X_train_pca, y_train['cod_label'])

# Avaliando a acurácia nos dados de teste transformados
test_accuracy_pca = tree_model_pca.score(X_test_pca, y_test['cod_label'])

print("Acurácia com PCA nos dados de treinamento:", round(train_accuracy_pca, 2))
print("Acurácia com PCA nos dados de teste:", round(test_accuracy_pca, 2))
print("Tempo de treinamento com PCA:", round(training_time, 2), "segundos")
print("-" * 60)


Acurácia com PCA nos dados de treinamento: 0.5
Acurácia com PCA nos dados de teste: 0.46
Tempo de treinamento com PCA: 0.13 segundos
------------------------------------------------------------
CPU times: total: 750 ms
Wall time: 563 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [21]:
%%time

# Lista de quantidades de componentes a serem testadas
n_components_list = [1, 2, 5, 10, 50]

for n_components in n_components_list:
    print(f"Testando com {n_components} componentes principais")
    
    # Realizando a Análise de Componentes Principais (PCA) com a quantidade atual de componentes
    pca = PCA(n_components=n_components)

    # Ajustando e transformando os dados de treinamento e teste com PCA
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    # Criando o modelo de Árvore de Decisão com as componentes PCA
    tree_model_pca = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)

    # Treinando o modelo com os dados de treinamento transformados
    start_time = time.time()
    tree_model_pca.fit(X_train_pca, y_train['cod_label'])
    training_time = time.time() - start_time

    # Avaliando a acurácia nos dados de treinamento transformados
    train_accuracy_pca = round(tree_model_pca.score(X_train_pca, y_train['cod_label']), 2)

    # Avaliando a acurácia nos dados de teste transformados
    test_accuracy_pca = round(tree_model_pca.score(X_test_pca, y_test['cod_label']), 2)

    print("Acurácia com PCA nos dados de treinamento:", train_accuracy_pca)
    print("Acurácia com PCA nos dados de teste:", test_accuracy_pca)
    print("Tempo de treinamento com PCA:", round(training_time, 2), "segundos")
    print()


Testando com 1 componentes principais
Acurácia com PCA nos dados de treinamento: 0.5
Acurácia com PCA nos dados de teste: 0.46
Tempo de treinamento com PCA: 0.11 segundos

Testando com 2 componentes principais
Acurácia com PCA nos dados de treinamento: 0.61
Acurácia com PCA nos dados de teste: 0.58
Tempo de treinamento com PCA: 0.13 segundos

Testando com 5 componentes principais
Acurácia com PCA nos dados de treinamento: 0.85
Acurácia com PCA nos dados de teste: 0.79
Tempo de treinamento com PCA: 0.11 segundos

Testando com 10 componentes principais
Acurácia com PCA nos dados de treinamento: 0.89
Acurácia com PCA nos dados de teste: 0.82
Tempo de treinamento com PCA: 0.3 segundos

Testando com 50 componentes principais
Acurácia com PCA nos dados de treinamento: 0.92
Acurácia com PCA nos dados de teste: 0.82
Tempo de treinamento com PCA: 1.17 segundos

CPU times: total: 5.19 s
Wall time: 4.43 s


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

> * Observando os resultados podemos notar que a acurácia aumentou à medida que o foram aumentados os números de componentes principais, sendo uma tendência comum em técnicas de redução de dimensionalidade como o PCA, a acurácia foi de 0,5 no treinamento e 0,46 no teste com apenas 1 componente principal, para 0,92 no treinamento e 0,82 no teste com 50 componentes principais.

> * Da mesma maneira, podemos observar que à medida que aumentamos os componentes principais, o tempo de processamento também aumentou, devido ao PCA envolver cálculos matriciais e transformações que utilizam intensivamente o poder computacional a medida que a dimensionalidade dos dados aumenta.